# Code used to generate the data set

In [26]:
import numpy as np
import os
import math
import shutil
# from moviepy.video.io.VideoFileClip import VideoFileClip
# from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [27]:
if not os.path.exists('videos'):
    shutil.unpack_archive('videos.zip', 'videos')


In [ ]:
#parse metadata
with open("data_unprocessed/metadata.txt") as file:
    lines =  np.array([line.rstrip().split(";") for line in file])

In [29]:
newpath = r'./temp' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

Ideally we would use a python library to download the youtube videos here. Unfortunatly a recent change to Youtubes cypher system has rendered all of these tools non-functional. Since our data set has a small number of long videos, we have opted to download them manually.

Currently the raw videos are not included in the repo, later I will include them in a zip archive if the github file limit allows it

In [30]:
from moviepy.config import FFMPEG_BINARY
import subprocess

def ffmpeg_extract_subclip(
    inputfile, start_time, end_time, outputfile=None, logger="bar"
):
    """Makes a new video file playing video file between two times.

    Parameters
    ----------

    inputfile : str
      Path to the file from which the subclip will be extracted.

    start_time : float
      Moment of the input clip that marks the start of the produced subclip.

    end_time : float
      Moment of the input clip that marks the end of the produced subclip.

    outputfile : str, optional
      Path to the output file. Defaults to
      ``<inputfile_name>SUB<start_time>_<end_time><ext>``.
    """
    if not outputfile:
        name, ext = os.path.splitext(inputfile)
        t1, t2 = [int(1000 * t) for t in [start_time, end_time]]
        outputfile = "%sSUB%d_%d%s" % (name, t1, t2, ext)

    cmd = [
        FFMPEG_BINARY,
        "-y",
        "-ss",
        "%0.2f" % start_time,
        "-i",
        inputfile,
        "-to",
        "%0.2f" % end_time,
        "-map",
        "0",
        "-vcodec",
        "copy",
        "-acodec",
        "copy",
        "-copyts",
        outputfile,
    ]
    subprocess.run(cmd)

In [31]:
links = np.unique(lines.transpose()[4])

i=0
while i < len(lines):
    
    if float(lines[i][5]) == float(lines[i+1][5])  and lines[i][4] == lines[i+1][4]:
        sample = lines[i if lines[i][6]> lines[i+1][6]  else i+1]
        start = float(lines[i][5])
        end = max(float(lines[i][6]), float(lines[i+1][6]))
        
        name = lines[i][0]+"&"+lines[i+1][0]
        i+=2
    else:
        sample = lines[i]
        start = float(sample[5])
        end = float(sample[6])
        name = sample[0]
        i+=1

    idx = np.where(links == sample[4])
    video = sample[4].split("=")[-1]
    ## If its the first video, remove once all have been downloaded
    # if idx[0][0] == 0:
    ffmpeg_extract_subclip("videos/"+video+".mp4", start, end, "cropped/" + name+".mp4")
        
       



Create a zip archive of the output so the videos can be commited to github

In [32]:
if os.path.isfile("videos_processed.zip"):
    os.remove("videos_processed.zip")
shutil.make_archive("videos_processed", 'zip', "cropped")

'c:\\Users\\mikeG\\Documents\\school\\cisc-452\\CISC-452-Group-22\\videos_processed.zip'

In [ ]:
# cropped\1_2018-07-12_1.mp4
# data_unprocessed\skeletons\11_2018-09-13_1.data
run = "11_2018-09-13_1"

import cv2
vidcap = cv2.VideoCapture('cropped\\'+run+'.mp4')
success,image = vidcap.read()
count = 1

with open("data_unprocessed/skeletons/"+run+".data") as file:
    data =  np.array([line.rstrip().split("#") for line in file])

frame_count =int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1

print(data)

offset = frame_count - int(data[-1][0])

for _ in range(offset):
    success,image = vidcap.read()

print ("Number of frames: ", int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1)
while success:
  if data[count][1] == "NULL":
      count += 1
      success,image = vidcap.read()
      continue
  cv2.imwrite("img/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

[['1' 'NULL']
 ['2' 'NULL']
 ['3' 'NULL']
 ['4' 'NULL']
 ['5'
  '749.8,703.1;759.6,660.4;756.3,584.9;720.2,591.4;730.1,673.5;756.3,735.9;739.9,584.9;743.2,486.4;743.2,476.5;749.8,420.7;766.2,440.4;785.9,466.7;776.0,486.4;707.1,483.1;697.2,492.9;723.5,443.7']
 ['6'
  '757.0,640.1;737.9,633.8;747.5,567.0;709.3,567.0;709.3,643.3;750.7,706.9;725.2,567.0;741.1,471.7;741.1,465.4;741.1,408.2;744.3,506.7;785.6,506.7;776.1,471.7;706.2,465.4;693.5,500.3;715.7,506.7']
 ['7'
  '741.6,663.0;729.5,617.8;732.5,554.6;696.4,551.6;690.4,617.8;732.5,657.0;714.5,554.6;726.5,452.2;726.5,443.2;729.5,392.0;750.6,485.3;783.7,491.3;762.7,452.2;687.4,452.2;681.3,485.3;708.4,485.3']
 ['8'
  '740.2,657.1;728.6,606.9;726.5,541.0;688.1,538.6;669.2,585.8;713.1,631.2;707.0,541.0;714.1,438.0;714.1,429.5;712.0,377.1;760.8,470.9;777.7,473.8;750.7,437.4;675.8,438.0;662.6,460.7;687.8,450.8']
 ['9'
  '738.8,651.1;727.7,595.9;720.5,527.4;679.8,525.6;648.1,553.8;693.7,605.4;699.5,527.4;701.6,423.8;701.6,415.9;694.5,362.2;771

In [ ]:
import json
import os
import sys

def print_to_log(*args, sep=' ', end='\n', file=sys.stdout, flush=False):
    """
    Custom print function that appends the output to 'log.log' and writes to the console.

    Args:
        *args: Values to be printed.
        sep (str): Separator between values (default: ' ').
        end (str): End character (default: '\n').
        file: File-like object to write to (default: sys.stdout).
        flush (bool): Whether to forcibly flush the stream.
    """
    message = sep.join(map(str, args)) + end  # Construct the message

    # Append to the log file
    with open("log.log", "a") as log_file:
        log_file.write(message)


def parse_metadata(metadata_path):
    metadata = {}
    with open(metadata_path, 'r') as f:
        for line in f:
            parts = line.strip().split(';')
            metadata[parts[0]] = {
                'id_climber': parts[1],
                'date': parts[2],
                'run_number': parts[3],
                'url': parts[4],
                'start': float(parts[5]),
                'end': float(parts[6]),
                'time_sec': float(parts[7]),
                'time_frames': int(parts[8]),
                'finished': int(parts[9]),
                'side': parts[10],
                'fps': float(parts[11])
            }
    return metadata


def parse_skeletons(skeletons_dir):
    skeleton_data = {}
    for file_name in os.listdir(skeletons_dir):
        # Split the file name to extract metadata
        parts = file_name.replace('.data', '').split('_')
        if len(parts) != 3:
            print(f"Skipping invalid file name: {file_name}")
            continue
        
        participant_number = parts[0]
        date = parts[1]  # Format: year-month-day
        run_number = parts[2]
        run_id = f"{participant_number}_{date}_{run_number}"

        if run_id not in skeleton_data:
            skeleton_data[run_id] = {
                "participant_number": participant_number,
                "date": date,
                "run_number": run_number,
                "frames": []
            }

        # Parse skeleton data for each frame
        with open(os.path.join(skeletons_dir, file_name), 'r') as f:
            for line in f:
                if "NULL" in line:
                    continue
                parts = line.strip().split('#')
                frame_number = int(parts[0])
                keypoints = parts[1].split(';')
                # Parse coordinates and add visibility (2) for each joint
                keypoints = [float(coord) for kp in keypoints for coord in kp.split(',')]
                keypoints_with_visibility = []
                for i in range(0, len(keypoints), 2):
                    keypoints_with_visibility.extend([keypoints[i], keypoints[i + 1], 2])  # Add visibility
                skeleton_data[run_id]["frames"].append({
                    "frame_number": frame_number,
                    "keypoints": keypoints_with_visibility
                })
    return skeleton_data


def generate_coco_json(images_dir, skeletons, output_json):
    coco_data = {
        'images': [],
        'annotations': [],
        'categories': [{
            'id': 1,
            'name': 'person',
            'keypoints': [f'j{i+1}' for i in range(16)],
            'skeleton': [[i, i+1] for i in range(1, 16)]  # Simple linear skeleton
        }]
    }

    annotation_id = 1
    for run_id, run_data in skeletons.items():
        participant_number = run_data["participant_number"]
        date = run_data["date"]
        run_number = run_data["run_number"]

        for frame in run_data["frames"]:
            frame_number = frame["frame_number"]
            image_id = len(coco_data["images"]) + 1
            image_file = os.path.join(images_dir, f"frame{frame_number}.png")
            
            coco_data["images"].append({
                "id": image_id,
                "file_name": image_file,
                "height": 720,  # Adjust based on actual image size
                "width": 1280,
                "participant_number": participant_number,
                "run_number": run_number,
                "date": date
            })

            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": 1,
                "keypoints": frame["keypoints"],
                "num_keypoints": 16,
                "participant_number": participant_number,
                "run_id": run_id
            })
            annotation_id += 1

    with open(output_json, 'w') as f:
        json.dump(coco_data, f, indent=4)

# Paths
images_dir = 'img'
skeletons_dir = 'data_unprocessed/skeletons'
metadata_file = 'data_unprocessed/metadata.txt'
output_json = 'climbData.json'
# Process
metadata = parse_metadata(metadata_file)
skeletons = parse_skeletons(skeletons_dir)
generate_coco_json(images_dir, skeletons, metadata, output_json)


Skipping invalid file name: 10_2020-11-21_1.data
Skipping invalid file name: 11_2018-07-28_1.data
Skipping invalid file name: 11_2018-09-13_1.data
Skipping invalid file name: 11_2018-09-13_2.data
Skipping invalid file name: 11_2018-10-28_1.data
Skipping invalid file name: 11_2019-04-12_1.data
Skipping invalid file name: 11_2019-04-12_2.data
Skipping invalid file name: 11_2019-04-12_3.data
Skipping invalid file name: 11_2019-05-03_2.data
Skipping invalid file name: 11_2019-07-05_1.data
Skipping invalid file name: 11_2019-07-05_2.data
Skipping invalid file name: 11_2019-07-12_1.data
Skipping invalid file name: 11_2019-07-12_3.data
Skipping invalid file name: 11_2019-07-12_4.data
Skipping invalid file name: 11_2019-08-17_1.data
Skipping invalid file name: 11_2019-10-05_1.data
Skipping invalid file name: 11_2019-10-05_2.data
Skipping invalid file name: 11_2019-10-05_4.data
Skipping invalid file name: 11_2019-10-19_1.data
Skipping invalid file name: 11_2019-10-19_2.data
Skipping invalid fil

TypeError: generate_coco_json() takes 3 positional arguments but 4 were given